Define all quests in the game

In [14]:
from collections import namedtuple
Quest = namedtuple("Quest", "name type requirements rewards")

QUESTS = [
    Quest("Orange Quest", "Warfare", {"orange": 2}, {"VP": 4}),
    Quest("Black Quest", "Skullduggery", {"orange": 2}, {"VP": 4}),
]

In [15]:
print(QUESTS)

[Quest(name='Orange Quest', type='Warfare', requirements={'orange': 2}, rewards={'VP': 4}), Quest(name='Black Quest', type='Skullduggery', requirements={'orange': 2}, rewards={'VP': 4})]


Demonstrate how to access quest data

In [17]:
orangeQuest = QUESTS[0]
print(orangeQuest)
print(orangeQuest.type)
print(orangeQuest.rewards)
print(orangeQuest.rewards["VP"])

Quest(name='Orange Quest', type='Warfare', requirements={'orange': 2}, rewards={'VP': 4})
Warfare
{'VP': 4}
4
